# Inverting for surface creep rate from InSAR profile data

Gareth Funning, University of California, Riverside

This notebook takes profiles through InSAR velocity data from ascending and descending viewing geometries, and solves for the surface offset rate by fitting a step function. 

## Contents

0. [Dependencies](#dep)
1. [InSAR velocity data](#InSAR)
2. [Fault data wrangling](#fault)
3. [Find average fault strike](#strike)
4. [Interpolate fault segment data](#interpolate)
5. [Projecting InSAR data onto a profile](#projecting)
6. [Fitting straight lines to the profiles](#fitting)
7. [The big loop](#bigloop)
8. [Other outputs](#other)
9. [References](#refs)

## <a name="dep"></a>0. Dependencies

This notebook makes use of a few dependencies. Make sure they're all installed before you get started, otherwise Jupyter will be sad...

Other than some fairly standard libraries, you will need to download [interparc.py](https://github.com/rsyi/python-lib/blob/master/interparc.py) to your working directory.

In [ ]:
# import your dependencies
import numpy as np
import pandas as pd
from interparc import interparc
import utm
import matplotlib.pyplot as plt
import pygmt

## <a name="InSAR"></a>1. InSAR velocity data 

In this exercise we are going to analyze pre-processed InSAR deformation velocities. These are supplied along with their corresponding line-of-sight (LOS) information. 

In [ ]:
# data file names, directories, and sign convention
asc_tr = 'A137'
dsc_tr = 'D042'
signconvention = -1   # 1 if data are in range change, -1 if they are ground displacement

asc_file = '../' + asc_tr + '/vel/ARIA_' + asc_tr + '_vel_ll.grd'
#asc_file = '../' + asc_tr + '_masked_ll.grd'
asc_azi = '../LOSfiles_CA/' + asc_tr + '/LOS/ARIA_' + asc_tr + '_azimuthAngle_ll.grd'
asc_inc = '../LOSfiles_CA/' + asc_tr + '/LOS/ARIA_' + asc_tr + '_incidenceAngle_ll.grd'

dsc_file = '../' + dsc_tr + '/vel/ARIA_' + dsc_tr + '_vel_ll.grd'
#dsc_file = '../'  + dsc_tr + '_masked_ll.grd'
dsc_azi = '../LOSfiles_CA/' + dsc_tr + '/LOS/ARIA_' + dsc_tr + '_azimuthAngle_ll.grd'
dsc_inc = '../LOSfiles_CA/' + dsc_tr + '/LOS/ARIA_' + dsc_tr + '_incidenceAngle_ll.grd'

print('ascending track ' + asc_tr + ', data file: ' + asc_file)
print('descending track ' + dsc_tr + ', data file: ' + dsc_file)

## <a name="fault"></a>2. Fault data wrangling

Let's load in some fault location data, and figure out some points that we can use for profiles.

In [ ]:
# some optional nonsense for prepping some fault data
!rm doc.kml
!unzip SanAndreas_shallow.kmz  # I crudely traced the fault with this Google Earth kmz line file
!gmt kml2gmt doc.kml |  awk '{if (NR>1) print $0}' | sort -n > SanAndreas_shallow.xy  # grab some coordinates from it!

In [ ]:
# some fault stuff
infault='SanAndreas_shallow.xy' # a file of long-lat coordinates that are on the fault of interest

# these two inputs can take multiple different values in array form
box_hw=np.array([2, 4])        # the (strike-perpendicular) half width of the boxes in km
mask_hw=np.array([0.2, 0.2])   # strike-perpendicular distance (in km) to mask out

# these are fixed for the whole fault
box_length=2            # (along-strike) length of the boxes in km
pt_spacing=0.1          # sample spacing (km)

buffer=0;    # x/y overlap between boxes

# some coordinate business
utm_zone = 10
utm_letter = 'S' # consult here if unsure: http://maps.unomaha.edu/Peterson/gis/notes/MapProjCoord.html

# here are some example coordinates
fault_data_ll=np.loadtxt(infault)

# transform fault coordinates into utm
fault_data_utm=utm.from_latlon(fault_data_ll[:,1],fault_data_ll[:,0],utm_zone,utm_letter)
eastings=fault_data_utm[0]
northings=fault_data_utm[1]

# extract line segments from the input vertices
# utm outputs horizontal arrays, which we will have to deal with
# np.vstack stacks them... -2 means to penultimate element, -1 means to last element
# and transpose to make it columns again
fault_segments=np.transpose(np.vstack((eastings[0:-1],northings[0:-1],eastings[1:],northings[1:])))

# calculate total line length by doing pythagoras and summing! 
fault_length=np.sum(np.sqrt(np.square(fault_segments[:,2]-fault_segments[:,0])
        +np.square(fault_segments[:,3]-fault_segments[:,1])))/1000

# how many fault nodes do we need?
nnodes = round(fault_length/(box_length))+1

# and what is the node spacing?
nspacing = fault_length/(nnodes-1)

print('Fault length ' + str(round(fault_length,1)) + 
      ' km, we will use ' + str(nnodes) + ' nodes')

## <a name="strike"></a>3. Find the average fault strike

I have had issues with projecting things into local strike-parallel velocity, so we will experiment with using just the average strike. Experimental!

In [ ]:
# we want to fit a best-fitting line to the fault vertices!
A=np.transpose(np.vstack((eastings,np.ones(np.shape(eastings)))))   # form the design matrix
ATAinv=np.linalg.inv(np.matmul(A.T,A))          # make and invert ATA in one go!
m=np.matmul(ATAinv,np.matmul(A.T,northings))    # m is the model vector, the gradient and intercept of the best fit line
av_str=np.degrees(np.arctan(1/m[0]))+180        # the arctan of the inverse gradient is the strike!

print('Average strike is {0:6.2f} degrees'.format(av_str))


## <a name="interpolate"></a>4. Interpolate fault segment data

We need to calculate some things about the fault.

In [ ]:
# next, interpolate the coordinates of yer fault

node_utm=interparc(nnodes,eastings,northings)
node_ll=utm.to_latlon(node_utm[:,0],node_utm[:,1],utm_zone,utm_letter)

seg_verts=np.transpose(np.vstack((node_utm[0:-1,0],node_utm[0:-1,1],node_utm[1:,0],node_utm[1:,1])))

# calculate strike
stk=np.degrees(np.arctan2(seg_verts[:,2]-seg_verts[:,0],seg_verts[:,3]-seg_verts[:,1]))

#
seg_centers_utm=np.array(([(seg_verts[:,2]+seg_verts[:,0])/2,(seg_verts[:,3]+seg_verts[:,1])/2]))
seg_centers_ll=utm.to_latlon(seg_centers_utm[0],seg_centers_utm[1],utm_zone,utm_letter)

# tot up the along-profile distance
seg_dist=np.arange(0,nnodes-1)*nspacing

# make a data frame to put things in    
fault_segs=pd.DataFrame({'longitude': seg_centers_ll[1], 'latitude': seg_centers_ll[0], 
                         'x': seg_centers_utm[0], 'y': seg_centers_utm[1], 'distance': seg_dist, 'strike': stk})

# and we'll sample the LOS information at the centers
fault_segs=pygmt.grdtrack(fault_segs,asc_azi,newcolname='asc_azi',interpolation="n")
fault_segs=pygmt.grdtrack(fault_segs,asc_inc,newcolname='asc_inc',interpolation="n")
fault_segs=pygmt.grdtrack(fault_segs,dsc_azi,newcolname='dsc_azi',interpolation="n")
fault_segs=pygmt.grdtrack(fault_segs,dsc_inc,newcolname='dsc_inc',interpolation="n")

# we'll ignore rows that contain zeros in the new data columns
fault_segs=fault_segs.loc[(fault_segs[['asc_azi']] != 0).any(axis=1)]
fault_segs=fault_segs.loc[(fault_segs[['dsc_azi']] != 0).any(axis=1)]

# and reindex the data frame after throwing out the blanks
fault_segs=fault_segs.reset_index(drop=True)

In [ ]:
fault_segs

## <a name="projecting"></a>5. Projecting InSAR data onto a profile

We will try and use the PyGMT command 'grdtrack' to sample these data files. grdtrack reads coordinates from text files or pandas data frames, and we will generate one of the latter. 

The first step is to ingest fault data, convert the locations into UTM (a rectilinear local coordinate system, which is more appropriate for this sort of analysis) and calculate the average strike:

Next, we will construct the profile coordinates. We'll pick the middle of the fault as our profile location. Once we specify dimensions of an area to sample, we will calculate the coordinates of a grid of points that we will use to sample the data files, project them back into geographical coordinates, and put them into a data frame. Simple (if a bit fiddly)!

In [ ]:
# let's pick a profile location and length
prof_num=21    # profile number
plen=1        # index of the profile length you want

prof_x=fault_segs.iloc[prof_num]["x"]    # x coordinate of profile center (in UTM) 
prof_y=fault_segs.iloc[prof_num]["y"]    # y coordinate 
fault_strike=fault_segs.iloc[prof_num]["strike"]

# unit vector in the along-strike direction
fault_dx=np.sin(np.radians(fault_strike))
fault_dy=np.cos(np.radians(fault_strike))

# unit vector in the along-profile direction (90 degrees from strike)
prof_dx=np.sin(np.radians(fault_strike-90))
prof_dy=np.cos(np.radians(fault_strike-90))

# let's make vectors of along-profile and cross-profile distances
p=np.linspace(-box_hw[plen],box_hw[plen],round(2*box_hw[plen]/pt_spacing)+1)  #  p = along-profile distance
q=np.linspace(-box_length/2,box_length/2,round(box_length/pt_spacing)+1)      # q = cross-profile distance
vp=np.linspace(-box_hw[plen],box_hw[plen],2)                # the p coordinates of the corners of the profile
vq=np.linspace(-box_length/2,box_length/2,2)                # the q coordinates of the corners of the profile

# we can make grids of points in (p,q) coordinates
P, Q = np.meshgrid(p,q)                 # all the points we want to sample
VP = np.array([[vp], [np.flip(vp)]])    # the vertices of the area being sampled
VQ = np.array([[vq], [vq]]).T

# transform these coordinates into UTM
X=prof_x+P*prof_dx*1000+Q*fault_dx*1000       # x coordinates (in UTM) of sample points 
Y=prof_y+P*prof_dy*1000+Q*fault_dy*1000       # y coordinates (in UTM) of sample points
VX=prof_x+VP*prof_dx*1000+VQ*fault_dx*1000    # x coordinates of sample region vertices
VY=prof_y+VP*prof_dy*1000+VQ*fault_dy*1000    # y coordinates of sample region vertices

# let's get the UTM coordinates for the profile line, too
px=prof_x+1000*prof_dx*vp
py=prof_y+1000*prof_dy*vp

# flattening grids into single columns,  transform these into lat-long
pt_locs_ll=utm.to_latlon(X.reshape(-1),Y.reshape(-1),utm_zone,utm_letter)
vt_locs_ll=utm.to_latlon(VX.reshape(-1),VY.reshape(-1),utm_zone,utm_letter)
pf_locs_ll=utm.to_latlon(px,py,utm_zone,utm_letter)
pf_center_ll=utm.to_latlon(prof_x,prof_y,utm_zone,utm_letter)

# let's make data frames for these
samp_pts=pd.DataFrame({'longitude': pt_locs_ll[1], 'latitude': pt_locs_ll[0], 'P': P.reshape(-1), 'Q': Q.reshape(-1)})
samp_vts=pd.DataFrame({'longitude': vt_locs_ll[1], 'latitude': vt_locs_ll[0], 'P': VP.reshape(-1), 'Q': VQ.reshape(-1)})

If you like, you can see what these data frames look like:

In [ ]:
# plot the profile and sample area on a map!

# set bounds
study_region=[-121.8,-120,35.5,37.0]  # [west, east, south, north]

# initiate a PyGMT figure 
fig = pygmt.Figure() 

# override some ugly (I think) defaults:
pygmt.config(FORMAT_GEO_MAP="ddd.x")

# make a colormap
pygmt.makecpt(cmap='polar', series=(-15, 15, 0.5), background="o")

# and let's get plotting!
fig.basemap(region=study_region, projection="M12c", frame=["a1f0.2","WeSn"])  # make a basemap frame
fig.grdimage(asc_file)                                                        # plot one data file
fig.coast(shorelines=["1/1p,navy","2/1p,navy"], borders=["1/0.5p,darkred"])   # coasts and borders
fig.text(text=asc_tr, position="BL", font="14p", offset="0.2c", justify="BL") # label the plot
fig.plot(x=samp_vts["longitude"],y=samp_vts["latitude"], pen="1p,black",close=True)   # sample box
fig.plot(x=node_ll[1],y=node_ll[0],pen="1.5p,gray50")              # fault trace

fig.shift_origin(xshift="14c")                                                # shift the origin
fig.basemap(region=study_region, projection="M12c", frame=["a1f0.2","weSn"])  # and let's make another one!
fig.grdimage(dsc_file)                                                        # plot the other file
fig.coast(shorelines=["1/1p,navy","2/1p,navy"], borders=["1/0.5p,darkred"])   # coasts and borders
fig.text(text=dsc_tr, position="BL", font="14p", offset="0.2c", justify="BL") # label this one too
fig.plot(x=samp_vts["longitude"],y=samp_vts["latitude"], pen="1p,black",close=True)   # sample box
fig.plot(x=node_ll[1],y=node_ll[0],pen="1.5p,gray50")              # fault trace

# and display!
fig.show(width=800)


Look good? Then we shall proceed with data sampling! As mentioned, we will use pygmt.grdtrack to do this...

In [ ]:
# here goes nothing!
samp_pts=pygmt.grdtrack(samp_pts,asc_file,newcolname=asc_tr,interpolation="n")  # add a column for the ascending data
samp_pts=pygmt.grdtrack(samp_pts,dsc_file,newcolname=dsc_tr,interpolation="n")  # add a column for the descending data

# we'll ignore rows that contain zeros in the two new data columns
samp_pts=samp_pts.loc[(samp_pts[[asc_tr, dsc_tr]] != 0).all(axis=1)]

That was easy! The effect is to add a couple of extra columns to 'samp_pts'. Let's have a look:

In [ ]:
samp_pts

And we can look at the values by plotting them with matplotlib:

In [ ]:
# initiate the plot
plt.figure(figsize=(12,6))

# plot the data as red inverted triangles with 2-sigma error bars
plt.scatter(samp_pts["P"], samp_pts[asc_tr], marker=".", label=asc_tr)  
plt.scatter(samp_pts["P"], samp_pts[dsc_tr], marker=".", label=dsc_tr)  
    
# add some labels
plt.xlabel("x (km)")
plt.ylabel("v (mm/yr)")
plt.legend()

plt.show()

In [ ]:
asc_azi=fault_segs.iloc[prof_num]["asc_azi"]
asc_inc=fault_segs.iloc[prof_num]["asc_inc"]
dsc_azi=fault_segs.iloc[prof_num]["dsc_azi"]
dsc_inc=fault_segs.iloc[prof_num]["dsc_inc"]

# calculate the unit LOS vectors (satellite pointing to ground target)
asc_los=np.array([np.sin(np.radians(asc_azi))*np.sin(np.radians(asc_inc)),
                  -np.cos(np.radians(asc_azi))*np.sin(np.radians(asc_inc)),
                  -np.cos(np.radians(asc_inc))])*signconvention

dsc_los=np.array([np.sin(np.radians(dsc_azi))*np.sin(np.radians(dsc_inc)),
                  -np.cos(np.radians(dsc_azi))*np.sin(np.radians(dsc_inc)),
                  -np.cos(np.radians(dsc_inc))])*signconvention

# report the sign convention
if signconvention==1:
    print('range change sign convention')
elif signconvention==-1:
    print('ground displacement sign convention')
else:
    print('you may have messed up this sign convention thing, to be honest')

# and report your LOS vector
print('ascending track LOS vector: [{0:5.3f}, {1:5.3f}, {2:5.3f}]'.format(asc_los[0],asc_los[1],asc_los[2]))
print('descending track LOS vector: [{0:5.3f}, {1:5.3f}, {2:5.3f}]'.format(dsc_los[0],dsc_los[1],dsc_los[2]))

## <a name="fitting"></a>6. Fitting straight lines to the profiles

We still have a few steps to go in order to estimate surface creep rate from the InSAR data. Next, we need to try and fit the step function to both profiles simultaneously. Could be fun!

We can try some forward models, first, and project them into the LOS directions we have just calculated. Recall that the 1D model deals with fault-parallel velocities, so we need to consider the direction of fault-parallel motion (the along-strike unit vector) when doing this $-$ we need to take the dot product between the along-strike unit vector and the LOS vector:

In [ ]:
# extract the window of data we want to fit
prof_1=samp_pts.loc[(samp_pts['P'] >= -box_hw[plen]) & (samp_pts['P'] <= -mask_hw[plen])]
prof_2=samp_pts.loc[(samp_pts['P'] >= mask_hw[plen]) & (samp_pts['P'] <= box_hw[plen])]

# set up the ascending data set inversion
A_asc=np.vstack((np.transpose(np.vstack((prof_1['P'],np.ones(np.shape(prof_1['P'])),np.zeros(np.shape(prof_1['P']))))),
                 np.transpose(np.vstack((prof_2['P'],np.zeros(np.shape(prof_2['P'])),np.ones(np.shape(prof_2['P'])))))))
d_asc=np.concatenate([prof_1[asc_tr],prof_2[asc_tr]])

# set up the descending data set inversion
A_dsc=np.vstack((np.transpose(np.vstack((prof_1['P'],np.ones(np.shape(prof_1['P'])),np.zeros(np.shape(prof_1['P']))))),
                 np.transpose(np.vstack((prof_2['P'],np.zeros(np.shape(prof_2['P'])),np.ones(np.shape(prof_2['P'])))))))
d_dsc=np.concatenate([prof_1[dsc_tr],prof_2[dsc_tr]])

# invert for ascending offset rate 
ATAinv_asc=np.linalg.inv(np.matmul(A_asc.T,A_asc))    # make and invert ATA in one go!
m_asc=np.matmul(ATAinv_asc,np.matmul(A_asc.T,d_asc))  # m_asc is the model vector

# misfit!
dprime_asc=np.matmul(A_asc,m_asc)
res_asc=d_asc-dprime_asc
rms_asc=np.sqrt(np.matmul(res_asc.T,res_asc)/np.size(res_asc))

# invert for descending offset rate 
ATAinv_dsc=np.linalg.inv(np.matmul(A_dsc.T,A_dsc))    # make and invert ATA in one go!
m_dsc=np.matmul(ATAinv_dsc,np.matmul(A_dsc.T,d_dsc))  # m_dsc is the model vector

# misfit!
dprime_dsc=np.matmul(A_dsc,m_dsc)
res_dsc=d_dsc-dprime_dsc
rms_dsc=np.sqrt(np.matmul(res_dsc.T,res_dsc)/np.size(res_dsc))

# collate and report the results
asc_offset=m_asc[2]-m_asc[1]
sigma_asc_offset=np.sqrt(ATAinv_asc[1,1]+ATAinv_asc[2,2])
dsc_offset=m_dsc[2]-m_dsc[1]
sigma_dsc_offset=np.sqrt(ATAinv_dsc[1,1]+ATAinv_dsc[2,2])
print("profile {0:d}".format(prof_num))
print("ascending offset rate {0:4.1f} +/- {1:3.1f} (2-sigma), descending offset rate {2:4.1f} +/- {3:3.1f} (2-sigma)"
      .format(asc_offset,2*sigma_asc_offset,dsc_offset,2*sigma_dsc_offset))
print("ascending rms {0:4.1f}, descending rms {1:4.1f}".format(rms_asc,rms_dsc))


# and let's estimate a creep rate! set up matrices and vectors:
A_creep=np.vstack(([fault_dx*asc_los[0]+fault_dy*asc_los[1], asc_los[2]],
                    [fault_dx*dsc_los[0]+fault_dy*dsc_los[1], dsc_los[2]]))
d_creep=np.vstack(([asc_offset],[dsc_offset]))
Einv_creep=np.vstack(([1/sigma_asc_offset**2, 0],[0, 1/sigma_dsc_offset**2]))

# and the normal equations, etc
ATAinv_creep=np.linalg.inv(np.matmul(np.matmul(A_creep.T,Einv_creep),A_creep))
ATd_creep=np.matmul(np.matmul(A_creep.T,Einv_creep),d_creep)

# and solve!
m_creep=np.matmul(ATAinv_creep,ATd_creep)

h_offset=m_creep[0]
sigma_h_offset=np.sqrt(ATAinv_creep[0,0])
v_offset=m_creep[1]
sigma_v_offset=np.sqrt(ATAinv_creep[1,1])
print("fault-parallel offset rate {0:5.1f} +/- {1:3.1f} (2-sigma), vertical offset rate {2:5.1f} +/- {3:3.1f} (2-sigma)"
      .format(h_offset[0],2*sigma_h_offset,v_offset[0],2*sigma_v_offset))

In [ ]:
# next, let's detrend our data and plot them

abox=[-box_hw[plen], box_hw[plen], -15, 10]
dbox=[-box_hw[plen], box_hw[plen], -5, 20]

fig = pygmt.Figure() 

# and let's get plotting!
fig.basemap(region=abox, projection="X12c/6c", 
            frame=['xa1f0.2+l"distance (km)"','ya10f2+l"los velocity (mm/yr)"',"WeSn"])  # make a basemap frame
fig.text(text='{0:s}, {1:4.1f} +/- {2:3.1f}'.format(asc_tr,asc_offset,2*sigma_asc_offset), position="TL", font="14p", 
         offset="0.2c/-0.2c", justify="TL") # label the plot
fig.text(text='rms: {0:4.1f}'.format(rms_asc), position="TR", font="14p", offset="-0.2c/-0.2c", justify="TR") # label the plot
fig.text(text="profile {0:d}".format(prof_num), position="BL", font="14p", offset="0.2c", justify="BL") # label the plot
fig.plot(x=samp_pts["P"],y=samp_pts[asc_tr]-m_asc[0]*samp_pts["P"]-m_asc[1], style="c0.1c", color="cyan")   # detrended velocities
fig.plot(x=[-box_hw[plen], -mask_hw[plen]],y=[0,0], pen='2p,black,-')
fig.plot(x=[-mask_hw[plen], 0],y=[0,0], pen='2p,black,.')
fig.plot(x=[0, mask_hw[plen]],y=[asc_offset,asc_offset], pen='2p,black,.')
fig.plot(x=[mask_hw[plen], box_hw[plen]],y=[asc_offset,asc_offset], pen='2p,black,-')


fig.shift_origin(xshift="14c")                                                # shift the origin
fig.basemap(region=dbox, projection="X12c/6c", frame=['xa1f0.2+l"distance (km)"','ya10f2','WeSn'])  # make a basemap frame
fig.text(text='{0:s}, {1:4.1f} +/- {2:3.1f}'.format(dsc_tr,dsc_offset,2*sigma_dsc_offset), position="TL", font="14p", 
         offset="0.2c/-0.2c", justify="TL") # label the plot
fig.text(text='rms: {0:4.1f}'.format(rms_dsc), position="TR", font="14p", offset="-0.2c/-0.2c", justify="TR") # label the plot
fig.text(text="creep: {0:4.1f} +/- {1:3.1f}".format(h_offset[0],2*sigma_h_offset), position="BR", font="14p", 
         offset="-0.2c/0.2c", justify="BR") # label the plot
fig.plot(x=samp_pts["P"], y=samp_pts[dsc_tr]-m_dsc[0]*samp_pts["P"]-m_dsc[1], style="c0.1c", color="orange")  # detrended velocities
fig.plot(x=[-box_hw[plen], -mask_hw[plen]], y=[0,0], pen='2p,black,-')
fig.plot(x=[-mask_hw[plen], 0], y=[0,0], pen='2p,black,.')
fig.plot(x=[0, mask_hw[plen]], y=[dsc_offset,dsc_offset], pen='2p,black,.')
fig.plot(x=[mask_hw[plen], box_hw[plen]],y=[dsc_offset,dsc_offset], pen='2p,black,-')


fig.show(width=800)

## <a name="bigloop"></a>7. The big loop

Having tested all the parts of the process: estimating geometry, constructing profiles, sampling data, fitting lines, inverting for creep rate... the next step is to apply the whole process to every profile, in a monster loop. We will store all the results in another pandas data frame, and output every profile plot as an image file. Stand by...

In [ ]:
plen=0       # choose which profile_length to use

print('profile half-width = {0:f} km'.format(box_hw[plen]))
print('mask half-width = {0:f} km'.format(mask_hw[plen]))

# make a data frame to store results
creep_rates=pd.DataFrame({'distance': fault_segs["distance"], 'strike': fault_segs["strike"], 
                          "{0:s}_offset".format(asc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "{0:s}_sigma".format(asc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "{0:s}_rms".format(asc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "{0:s}_offset".format(dsc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "{0:s}_sigma".format(dsc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "{0:s}_rms".format(dsc_tr): np.zeros(np.shape(fault_segs["strike"])),
                          "fp_offset": np.zeros(np.shape(fault_segs["strike"])),
                          "fp_sigma": np.zeros(np.shape(fault_segs["strike"])),
                          "v_offset": np.zeros(np.shape(fault_segs["strike"])),
                          "v_sigma": np.zeros(np.shape(fault_segs["strike"])) })

# for profile plotting later on
abox=[-box_hw[plen], box_hw[plen], -15, 10]
dbox=[-box_hw[plen], box_hw[plen], -5, 20]

# let's do the big loop!
for i, row in fault_segs.iterrows():
    
    prof_num=i
    print("profile {0:d}".format(prof_num))
    
    prof_x=fault_segs.iloc[prof_num]["x"]    # x coordinate of profile center (in UTM) 
    prof_y=fault_segs.iloc[prof_num]["y"]    # y coordinate 
    fault_strike=fault_segs.iloc[prof_num]["strike"]
    as_distance=fault_segs.iloc[prof_num]["distance"] 

    # unit vector in the along-strike direction
    fault_dx=np.sin(np.radians(fault_strike))
    fault_dy=np.cos(np.radians(fault_strike))

    # unit vector in the along-profile direction (90 degrees from strike)
    prof_dx=np.sin(np.radians(fault_strike-90))
    prof_dy=np.cos(np.radians(fault_strike-90))

    # let's make vectors of along-profile and cross-profile distances
    p=np.linspace(-box_hw[plen],box_hw[plen],round(2*box_hw[plen]/pt_spacing)+1)    #  p = along-profile distance
    q=np.linspace(-box_length/2,box_length/2,round(box_length/pt_spacing)+1)        # q = cross-profile distance
    vp=np.linspace(-box_hw[plen],box_hw[plen],2)                # the p coordinates of the corners of the profile
    vq=np.linspace(-box_length/2,box_length/2,2)                # the q coordinates of the corners of the profile

    # we can make grids of points in (p,q) coordinates
    P, Q = np.meshgrid(p,q)                 # all the points we want to sample
    VP = np.array([[vp], [np.flip(vp)]])    # the vertices of the area being sampled
    VQ = np.array([[vq], [vq]]).T

    # transform these coordinates into UTM
    X=prof_x+P*prof_dx*1000+Q*fault_dx*1000       # x coordinates (in UTM) of sample points 
    Y=prof_y+P*prof_dy*1000+Q*fault_dy*1000       # y coordinates (in UTM) of sample points
    VX=prof_x+VP*prof_dx*1000+VQ*fault_dx*1000    # x coordinates of sample region vertices
    VY=prof_y+VP*prof_dy*1000+VQ*fault_dy*1000    # y coordinates of sample region vertices

    # let's get the UTM coordinates for the profile line, too
    px=prof_x+1000*prof_dx*vp
    py=prof_y+1000*prof_dy*vp

    # flattening grids into single columns,  transform these into lat-long
    pt_locs_ll=utm.to_latlon(X.reshape(-1),Y.reshape(-1),utm_zone,utm_letter)
    vt_locs_ll=utm.to_latlon(VX.reshape(-1),VY.reshape(-1),utm_zone,utm_letter)
    pf_locs_ll=utm.to_latlon(px,py,utm_zone,utm_letter)
    pf_center_ll=utm.to_latlon(prof_x,prof_y,utm_zone,utm_letter)

    # let's make data frames for these
    samp_pts=pd.DataFrame({'longitude': pt_locs_ll[1], 'latitude': pt_locs_ll[0], 'P': P.reshape(-1), 'Q': Q.reshape(-1)})
    samp_vts=pd.DataFrame({'longitude': vt_locs_ll[1], 'latitude': vt_locs_ll[0], 'P': VP.reshape(-1), 'Q': VQ.reshape(-1)})

    # here goes nothing!
    samp_pts=pygmt.grdtrack(samp_pts,asc_file,newcolname=asc_tr,interpolation="n")  # add a column for the ascending data
    samp_pts=pygmt.grdtrack(samp_pts,dsc_file,newcolname=dsc_tr,interpolation="n")  # add a column for the descending data

    # we'll ignore rows that contain zeros in the two new data columns
    samp_pts=samp_pts.loc[(samp_pts[[asc_tr, dsc_tr]] != 0).all(axis=1)]
    
    # azimuths and incidences
    asc_azi=fault_segs.iloc[prof_num]["asc_azi"]
    asc_inc=fault_segs.iloc[prof_num]["asc_inc"]
    dsc_azi=fault_segs.iloc[prof_num]["dsc_azi"]
    dsc_inc=fault_segs.iloc[prof_num]["dsc_inc"]

    # calculate the unit LOS vectors (satellite pointing to ground target)
    asc_los=np.array([np.sin(np.radians(asc_azi))*np.sin(np.radians(asc_inc)),
                  -np.cos(np.radians(asc_azi))*np.sin(np.radians(asc_inc)),
                  -np.cos(np.radians(asc_inc))])*signconvention

    dsc_los=np.array([np.sin(np.radians(dsc_azi))*np.sin(np.radians(dsc_inc)),
                  -np.cos(np.radians(dsc_azi))*np.sin(np.radians(dsc_inc)),
                  -np.cos(np.radians(dsc_inc))])*signconvention

    # extract the window of data we want to fit
    prof_1=samp_pts.loc[(samp_pts['P'] >= -box_hw[plen]) & (samp_pts['P'] <= -mask_hw[plen])]
    prof_2=samp_pts.loc[(samp_pts['P'] >= mask_hw[plen]) & (samp_pts['P'] <= box_hw[plen])]

    # sanity check for data coverage
    if prof_1.empty:
        print('prof_1 is empty, skipping')
        continue
    if prof_2.empty:
        print('prof_2 is empty, skipping')
        continue
    
    # set up the ascending data set inversion
    A_asc=np.vstack((np.transpose(np.vstack((prof_1['P'],np.ones(np.shape(prof_1['P'])),np.zeros(np.shape(prof_1['P']))))),
                 np.transpose(np.vstack((prof_2['P'],np.zeros(np.shape(prof_2['P'])),np.ones(np.shape(prof_2['P'])))))))
    d_asc=np.concatenate([prof_1[asc_tr],prof_2[asc_tr]])

    # set up the descending data set inversion
    A_dsc=np.vstack((np.transpose(np.vstack((prof_1['P'],np.ones(np.shape(prof_1['P'])),np.zeros(np.shape(prof_1['P']))))),
                 np.transpose(np.vstack((prof_2['P'],np.zeros(np.shape(prof_2['P'])),np.ones(np.shape(prof_2['P']))))))) 
    d_dsc=np.concatenate([prof_1[dsc_tr],prof_2[dsc_tr]])

    # invert for ascending offset rate 
    ATAinv_asc=np.linalg.inv(np.matmul(A_asc.T,A_asc))    # make and invert ATA in one go!
    m_asc=np.matmul(ATAinv_asc,np.matmul(A_asc.T,d_asc))  # m_asc is the model vector

    # misfit!
    dprime_asc=np.matmul(A_asc,m_asc)
    res_asc=d_asc-dprime_asc
    rms_asc=np.sqrt(np.matmul(res_asc.T,res_asc)/np.size(res_asc))

    # invert for descending offset rate 
    ATAinv_dsc=np.linalg.inv(np.matmul(A_dsc.T,A_dsc))    # make and invert ATA in one go!
    m_dsc=np.matmul(ATAinv_dsc,np.matmul(A_dsc.T,d_dsc))  # m_dsc is the model vector
    
    # misfit!
    dprime_dsc=np.matmul(A_dsc,m_dsc)
    res_dsc=d_dsc-dprime_dsc
    rms_dsc=np.sqrt(np.matmul(res_dsc.T,res_dsc)/np.size(res_dsc))

    # collate and report the results
    asc_offset=m_asc[2]-m_asc[1]
    sigma_asc_offset=np.sqrt(ATAinv_asc[1,1]+ATAinv_asc[2,2])
    dsc_offset=m_dsc[2]-m_dsc[1]
    sigma_dsc_offset=np.sqrt(ATAinv_dsc[1,1]+ATAinv_dsc[2,2])
    print("ascending offset rate {0:4.1f} +/- {1:3.1f} (2-sigma), descending offset rate {2:4.1f} +/- {3:3.1f} (2-sigma)"
      .format(asc_offset,2*sigma_asc_offset,dsc_offset,2*sigma_dsc_offset))


    # and let's estimate a creep rate! set up matrices and vectors:
    A_creep=np.vstack(([fault_dx*asc_los[0]+fault_dy*asc_los[1], asc_los[2]],
                    [fault_dx*dsc_los[0]+fault_dy*dsc_los[1], dsc_los[2]]))
    d_creep=np.vstack(([asc_offset],[dsc_offset]))
    Einv_creep=np.vstack(([1/sigma_asc_offset**2, 0],[0, 1/sigma_dsc_offset**2]))

    # and the normal equations, etc
    ATAinv_creep=np.linalg.inv(np.matmul(np.matmul(A_creep.T,Einv_creep),A_creep))
    ATd_creep=np.matmul(np.matmul(A_creep.T,Einv_creep),d_creep)

    # and solve!
    m_creep=np.matmul(ATAinv_creep,ATd_creep)

    h_offset=m_creep[0]
    sigma_h_offset=np.sqrt(ATAinv_creep[0,0])
    v_offset=m_creep[1]
    sigma_v_offset=np.sqrt(ATAinv_creep[1,1])
    print("fault-parallel offset rate {0:5.1f} +/- {1:3.1f} (2-sigma), vertical offset rate {2:5.1f} +/- {3:3.1f} (2-sigma)"
      .format(h_offset[0],2*sigma_h_offset,v_offset[0],2*sigma_v_offset))
    
    
    # finally, plot it
    fig = pygmt.Figure() 
    
    fig.basemap(region=abox, projection="X12c/6c", 
            frame=['xa1f0.2+l"distance (km)"','ya10f2+l"los velocity (mm/yr)"',"WeSn"])  # make a basemap frame
    fig.text(text='{0:s}, {1:4.1f} +/- {2:3.1f}'.format(asc_tr,asc_offset,2*sigma_asc_offset), position="TL", font="14p", 
         offset="0.2c/-0.2c", justify="TL") # label the plot
    fig.text(text='rms: {0:4.1f}'.format(rms_asc), position="TR", font="14p", offset="-0.2c/-0.2c", justify="TR") 
    fig.text(text="d = {0:6.2f} km".format(as_distance), position="BL", 
             font="14p", offset="0.2c", justify="BL") # label the plot
    fig.plot(x=samp_pts["P"], y=samp_pts[asc_tr]-m_asc[0]*samp_pts["P"]-m_asc[1], style="c0.1c", color="cyan")   # detrended vels
    fig.plot(x=[-box_hw[plen], -mask_hw[plen]], y=[0,0], pen='2p,black,-')
    fig.plot(x=[-mask_hw[plen], 0], y=[0,0], pen='2p,black,.')
    fig.plot(x=[0, mask_hw[plen]], y=[asc_offset,asc_offset], pen='2p,black,.')
    fig.plot(x=[mask_hw[plen],box_hw[plen]], y=[asc_offset,asc_offset], pen='2p,black,-')


    fig.shift_origin(xshift="14c")                                                # shift the origin
    fig.basemap(region=dbox, projection="X12c/6c", frame=['xa1f0.2+l"distance (km)"','ya10f2','WeSn'])  # make a basemap frame
    fig.text(text='{0:s}, {1:4.1f} +/- {2:3.1f}'.format(dsc_tr,dsc_offset,2*sigma_dsc_offset), position="TL", font="14p", 
         offset="0.2c/-0.2c", justify="TL") # label the plot
    fig.text(text='rms: {0:4.1f}'.format(rms_dsc), position="TR", font="14p", offset="-0.2c/-0.2c", justify="TR") 
    fig.text(text="creep: {0:4.1f} +/- {1:3.1f}".format(h_offset[0],2*sigma_h_offset), position="BR", font="14p", 
         offset="-0.2c/0.2c", justify="BR") # label the plot
    fig.plot(x=samp_pts["P"], y=samp_pts[dsc_tr]-m_dsc[0]*samp_pts["P"]-m_dsc[1], style="c0.1c", color="orange")  # detrended vels
    fig.plot(x=[-box_hw[plen],-mask_hw[plen]], y=[0,0], pen='2p,black,-')
    fig.plot(x=[-mask_hw[plen],0], y=[0,0], pen='2p,black,.')
    fig.plot(x=[0,mask_hw[plen]], y=[dsc_offset,dsc_offset], pen='2p,black,.')
    fig.plot(x=[mask_hw[plen],box_hw[plen]], y=[dsc_offset,dsc_offset], pen='2p,black,-')


    figfile="profile_{0:s}_{1:s}_distance_{2:06.2f}_{3:f}_{4:f}.png".format(asc_tr,dsc_tr,as_distance,box_hw[plen],mask_hw[plen])
    #print(figfile)
    #fig.show(width=800)
    fig.savefig(figfile)
    
    # and record the results
    creep_rates.loc[i]["{0:s}_offset".format(asc_tr)]=asc_offset
    creep_rates.loc[i]["{0:s}_sigma".format(asc_tr)]=sigma_asc_offset
    creep_rates.loc[i]["{0:s}_rms".format(asc_tr)]=rms_asc
    creep_rates.loc[i]["{0:s}_offset".format(dsc_tr)]=dsc_offset
    creep_rates.loc[i]["{0:s}_sigma".format(dsc_tr)]=sigma_dsc_offset
    creep_rates.loc[i]["{0:s}_rms".format(dsc_tr)]=rms_dsc
    creep_rates.loc[i]["fp_offset"]=h_offset[0]
    creep_rates.loc[i]["fp_sigma"]=sigma_h_offset
    creep_rates.loc[i]["v_offset"]=v_offset[0]
    creep_rates.loc[i]["v_sigma"]=sigma_v_offset

# let's save these as a text file!
creep_rates.to_csv('{0:s}_{1:s}_creep_rates_{2:f}_{3:f}.dat'.format(asc_tr,dsc_tr,box_hw[plen],mask_hw[plen]),
                   sep=" ",index=False)

In [ ]:
creep_rates

In [ ]:
# let's plot the output

# (you may need to change these bounding boxes based on the fault length and profile values)

fp_box=[0, 200, -5, 35]
v_box=[0, 200, -5, 15]
asc_box=[0, 200, -15, 10]
dsc_box=[0, 200, -3, 22]

fig = pygmt.Figure() 


fig.basemap(region=fp_box, projection="X24c/5c", 
            frame=['xa10f2','ya10f2+l"creep rate (mm/yr)"',"Wesn"]) 
fig.plot(data=creep_rates[["distance","fp_offset","fp_sigma"]].values, error_bar="y+pfaint,gray50", style="t0.4c", 
         color="darkred",pen="faint", no_clip=True)   # creep rates
    
fig.shift_origin(yshift="-3.3c")                                                # shift the origin
fig.basemap(region=v_box, projection="X24c/2.5c", 
            frame=['xa10f2','ya10f2+l"vertical (mm/yr)"',"Wesn"]) 
fig.plot(data=creep_rates[["distance","v_offset","v_sigma"]].values, error_bar="y+pfaint,gray50", style="t0.4c", 
         color="hotpink",pen="faint", no_clip=True)   # creep rates


fig.shift_origin(yshift="-3.8c")                                                # shift the origin
fig.basemap(region=asc_box, projection="X24c/3c", 
            frame=['xa10f2','ya10f2+l"asc (mm/yr)"',"Wesn"]) 
fig.plot(data=creep_rates[["distance","{0:s}_offset".format(asc_tr),"{0:s}_sigma".format(asc_tr)]].values, 
         error_bar="y+pfaint,gray50", style="s0.4c", color="cyan", pen="faint", no_clip=True)   # offset rates

fig.shift_origin(yshift="-3.8c")                                                # shift the origin
fig.basemap(region=dsc_box, projection="X24c/3c", 
            frame=['xa10f2+l"distance from San Juan Bautista (km)"','ya10f2+l"dsc (mm/yr)"',"WeSn"]) 
fig.plot(data=creep_rates[["distance","{0:s}_offset".format(dsc_tr),"{0:s}_sigma".format(dsc_tr)]].values, 
         error_bar="y+pfaint,gray50", style="s0.4c", color="orange", pen="faint", no_clip=True)   # offset rates


#fig.text(text='{0:s}, {1:4.1f} +/- {2:3.1f}'.format(asc_tr,asc_offset,2*sigma_asc_offset), position="TL", font="14p", 
#         offset="0.2c/-0.2c", justify="TL") # label the plot
#fig.text(text="profile {0:d}".format(prof_num), position="BL", font="14p", offset="0.2c", justify="BL") # label the plot
    
#fig.plot([-box_hw, -mask_hw],[0,0], pen='2p,black,-')
#fig.plot([-mask_hw, 0],[0,0], pen='2p,black,.')
#fig.plot([0, mask_hw],[asc_offset,asc_offset], pen='2p,black,.')
#fig.plot([mask_hw, box_hw],[asc_offset,asc_offset], pen='2p,black,-')

fig.savefig('summary_creep_rate_{0:s}_{1:s}.png'.format(asc_tr,dsc_tr))
fig.show(width=800)

## <a name="other"></a>8. Other outputs

Useful for plotting purposes. 

In [ ]:
# we probably want to output the profile lines 

prof_dx=np.sin(np.radians(fault_strike-90))
prof_dy=np.cos(np.radians(fault_strike-90))


# the end coordinates in utm
x1_utm=fault_segs["x"]-(box_hw*1000)*np.sin(np.radians(fault_segs["strike"]-90))
y1_utm=fault_segs["y"]-(box_hw*1000)*np.cos(np.radians(fault_segs["strike"]-90))
x2_utm=fault_segs["x"]+(box_hw*1000)*np.sin(np.radians(fault_segs["strike"]-90))
y2_utm=fault_segs["y"]+(box_hw*1000)*np.cos(np.radians(fault_segs["strike"]-90))

# project!
x1y1_ll=utm.to_latlon(x1_utm,y1_utm,utm_zone,utm_letter)
x2y2_ll=utm.to_latlon(x2_utm,y2_utm,utm_zone,utm_letter)

# new dataframe
profile_lines=pd.DataFrame({'x1': x1y1_ll[1], 'y1': x1y1_ll[0], 'xc': fault_segs["longitude"], 'yc': fault_segs["latitude"], 
                            'x2': x2y2_ll[1], 'y2': x2y2_ll[0], 'distance': fault_segs["distance"], 
                            'strike': fault_segs["strike"]})

# and write it out
prof_lines_file='profile_lines_{0:s}_{1:s}.xy'.format(asc_tr,dsc_tr)
profile_lines.to_csv(prof_lines_file,sep=" ",index=False)

In [ ]:
# this doesn't seem to work
!awk '{if (NR>1) printf("> -L\"%06.2f\" -T\"profile %02d\"\n %f %f\n %f %f\n", $7, NR-2, $1, $2, $5, $6)}' $prof_lines_file | gmt gmt2kml -Fl -Wthick,white > $prof_lines_file.kml

## <a name="refs"></a>9. References

The method used in this notebook is based on that of [Lin and Funning, 2017](https://drive.google.com/file/d/1Ji_AZCl7SJNUMAIrUBC5GApPwBggbKhZ/view), although the exact math for computing the unit pointing vector is a little different as Sentinel-1 and ISCE made it more complicated...